In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import math

from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.metrics import mean_squared_error, r2_score
import surprise as sp
import lightgbm
import xgboost
from random import randint

import src.surprise_model
import src.lightgbm_model
from src.utils import split_in, extended_describe

## Predictions

### LGB con modelos

In [4]:
df_users = pd.read_csv('../data/01_raw/usuarios.csv')
df_books = pd.read_csv('../data/01_raw/libros.csv')
df_train = pd.read_csv('../data/02_intermediate/opiniones_train_modelos.csv')
df_test = pd.read_csv('../data/02_intermediate/opiniones_test_modelos.csv')

In [9]:
# Separo mi train & test
X_train, X_test, y_train, y_test = train_test_split(
    df_train.drop(["puntuacion"], axis=1),
    df_train.puntuacion,
    test_size=0.2,
    random_state=0,
)

In [14]:
a = pd.read_csv("../data/03_primary/opiniones_train_final.csv")
b = pd.read_csv("../data/03_primary/opiniones_test_final.csv")

In [169]:
!kaggle competitions submit -c prediccion-de-opiniones-de-libros-fcen-2020 -f submission_lgbm.csv -m "Lgbm con modelos de surprise"

100%|██████████████████████████████████████| 92.6k/92.6k [00:02<00:00, 33.5kB/s]
Successfully submitted to Predicción de opiniones de libros - FCEN 2020

Resultado a mejorar en Accuracy in test

Algoritmo | RMSE | Notas
- | - | -
SVD | 1.584 | NA
KNN | 1.683 | NA
LGBM | 1.555 | Con dos columnas de modelos de SVD y KNN
LGBM | 1.674 | Con genero y anio
LGBM | 1.323 | Con promedios -> esta overfitteando